In [1]:
from __future__ import print_function
import matplotlib
matplotlib.use('Qt5Agg')

from pylab import *
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import networkx as nx
import random as rd

threshold = 3

def init():
    global g, positions, s
    g = nx.karate_club_graph()
    for i in g:
        if random() < .5: 
            g.node[i]['pol'] = 1
        else:
            g.node[i]['pol'] = -1
        if random() < .5: 
            g.node[i]['eco'] = 1
        else:
            g.node[i]['eco'] = -1
    positions = nx.kamada_kawai_layout(g)
    
def update():
    global g, labels
    while True:
        speaker = rd.choice(list(g.nodes()))
        if not list(g.neighbors(speaker)):
            continue
        listener = rd.choice(list(g.neighbors(speaker)))
        break
    if random() < .5:
        update_pol(speaker, listener)
    else: 
        update_eco(speaker, listener)
    to_remove = []
    # listing the stubborn nodes that delete connections
    for p1 in g.nodes:
        pol1, eco1 = g.node[p1]['pol'], g.node[p1]['eco']
        for p2 in g.neighbors(p1):
            pol2, eco2 = g.node[p2]['pol'], g.node[p2]['eco']
            if abs(pol1) == threshold and abs(pol2) == threshold:
                if pol1*pol2<0 or eco1*eco2<0:
                    if (p2, p1) not in to_remove:
                        to_remove.append((p1, p2))
    for pair in list(set(to_remove)):
        g.remove_edge(*pair)            
    labels={}
    for i in g:
        labels[i]= str(int(g.node[i]['pol']))+","+str(int(g.node[i]['eco']))
    return speaker, listener

def update_pol(speaker, listener):
    global g
    l_pol = g.node[listener]['pol']
    s_pol = g.node[speaker]['pol']
    if abs( l_pol ) < threshold:
        if s_pol != 0:
            g.node[listener]['pol'] = l_pol + (s_pol)/abs(s_pol)
        elif l_pol != 0:
            g.node[listener]['pol'] = l_pol - (l_pol)/abs(l_pol)
        else:
            pass

def update_eco(speaker, listener):
    ## update food
    global g 
    l_eco = g.node[listener]['eco']
    s_eco = g.node[speaker]['eco']
    if abs( l_eco ) < threshold:
        if s_eco != 0:
            g.node[listener]['eco'] = l_eco + (s_eco)/abs(s_eco)
        elif l_eco != 0:
            g.node[listener]['eco'] = l_eco - (l_eco)/abs(l_eco)
        else:
            pass

def init_fig():
    global fig, viz
    fig = figure()
    viz_pol = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 500, edge_color='w',
                  node_color = [choose_pol_color(g.nodes[n]['pol']) for n in g.nodes])
    viz_eco = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 150, edgelist = [],
                  node_color = [choose_eco_color(g.nodes[n]['eco']) for n in g.nodes])
    fig.set_facecolor("#000000")

def choose_pol_color(pol):
    val = (pol+threshold)/2/threshold
    return plt.cm.seismic(val)

def choose_eco_color(eco):
    val = (eco+threshold)/2/threshold
    return plt.cm.seismic(val)

    
def observe(s=None, l=None):
    cla()
    global g, labels
    #cla()
    def get_edge_color(edge, s, l):
        if edge == (s, l) or edge == (l, s):
            return 'r'
        else:
            return 'w'
    if s and l:
        edge_colors = [get_edge_color(edge, s, l) for edge in g.edges]
    else:
        edge_colors = 'w'
    viz_pol = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 500, edge_color=edge_colors,
                  node_color = [choose_pol_color(g.nodes[n]['pol']) for n in g.nodes])
    viz_eco = nx.draw(g, with_labels=False, pos = positions,
                  node_size = 150, edgelist=[],
                  node_color = [choose_eco_color(g.nodes[n]['eco']) for n in g.nodes])
    nx.draw_networkx_labels(g,positions,labels,font_weight='bold',
                            font_color=plt.cm.viridis(0.95), font_size=18)
    fig = plt.gcf()
    fig.set_facecolor('k')
    return fig

init()
init_fig()
step = 5000
def step(t):
    s, l = update()
    observe(s, l)
    return [fig]

ani = FuncAnimation(fig, step, interval=0)
show()